# DeepESN <br/>
https://github.com/gallicch/DeepRC-TF

In [1]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input, Reshape, Dropout, Bidirectional, LSTM, GRU
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, multilabel_confusion_matrix, plot_confusion_matrix, classification_report, coverage_error

from DeepRC import *

In [2]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [3]:
df = pd.read_csv("data/train.csv")
df['comment_text'] = df['comment_text'].map(lambda com : clean_text(com))

categories = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
num_classes = len(categories)
train, valid = train_test_split(df, random_state=42, test_size=0.33, shuffle=True)

In [4]:
df_test = pd.read_csv('data/test.csv')
df['comment_text'] = df['comment_text'].map(lambda com : clean_text(com))
cols = df_test.columns
label_cols = list(cols[2:])

test_labels_df = pd.read_csv('data/test_labels.csv')
df_test = df_test.merge(test_labels_df, on='id', how='left')
test_label_cols = list(df_test.columns[2:])
print('Null values: ', df_test.isnull().values.any()) #should not be any null sentences or labels
print('Same columns between train and test: ', label_cols == test_label_cols) #columns should be the same
print(f"Number of rows original: {df_test.shape[0]}")

df_test = df_test[~df_test[test_label_cols].eq(-1).any(axis=1)] #remove irrelevant rows/comments with -1 values
df_test['one_hot_labels'] = list(df_test[test_label_cols].values)
print(f"Number of rows after: {df_test.shape[0]}")
df_test.head()

Null values:  False
Same columns between train and test:  False
Number of rows original: 153164
Number of rows after: 63978


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,one_hot_labels
5,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
7,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"
14,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0]"


In [60]:
X_train = train.comment_text
y_train = train[categories]
X_valid = valid.comment_text
y_valid = valid[categories]
X_test = df_test.comment_text.values
y_test = df_test[categories].values

tokenizer = Tokenizer(num_words=10)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_matrix(X_train, mode='tfidf')
X_valid = tokenizer.texts_to_matrix(X_valid, mode='tfidf')
X_test = tokenizer.texts_to_matrix(X_test, mode='tfidf')

X_train = tf.expand_dims(X_train, axis=2)
X_valid = tf.expand_dims(X_valid, axis=2)

y_train = y_train.values
y_valid = y_valid.values

print('training set',X_train.shape,y_train.shape)
print('validation set',X_valid.shape,y_valid.shape)
print('test set',X_test.shape,y_test.shape)

training set (106912, 10, 1) (106912, 6)
validation set (52659, 10, 1) (52659, 6)
test set (63978, 10) (63978, 6)


In [95]:
reservoir_units = 200
batch_size = 128

inputs = Input(shape=(X_train.shape[1],1), name='inputs')
reservoir = SimpleDeepReservoirLayer(units=reservoir_units, layers=10, trainable=False, name='reservoir')(inputs)

#reservoir = Reshape((1,reservoir_units), input_shape=(reservoir_units,))(reservoir)
#readout = Bidirectional(GRU(10), merge_mode='sum')(reservoir)

readout = Dense(100, activation='relu')(reservoir)
readout = Dropout(0.1)(readout)
readout = Dense(50, activation='relu')(readout)
readout = Dropout(0.1)(readout)
readout = Dense(num_classes, activation='sigmoid', name='readout')(readout)

model = Model(inputs, readout, name='DeepESN')

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=[
                    'accuracy',
                    'binary_accuracy', 
                    tf.keras.metrics.AUC(multi_label=True),
                    tf.keras.metrics.Precision(),
                    tf.keras.metrics.Recall()
              ])
model.summary()

Model: "DeepESN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 10, 1)]           0         
_________________________________________________________________
reservoir (SimpleDeepReservo (None, 200)               0         
_________________________________________________________________
dense_69 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_68 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_70 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_69 (Dropout)         (None, 50)                0         
_________________________________________________________________
readout (Dense)              (None, 6)                 306 

In [96]:
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=1, verbose=1, validation_data=(X_valid,y_valid))

836/836 [==============================] - 40s 45ms/step - loss: 0.1660 - accuracy: 0.6692 - binary_accuracy: 0.9496 - auc_39: 0.6040 - precision_39: 0.0430 - recall_39: 0.0150 - val_loss: 0.1308 - val_accuracy: 0.9937 - val_binary_accuracy: 0.9629 - val_auc_39: 0.7145 - val_precision_39: 0.4759 - val_recall_39: 0.0076


In [97]:
scores = model.evaluate(X_test, y_test, verbose=2)
print(f"Test {model.metrics_names}: {scores}")
scores = model.evaluate(X_valid, y_valid, verbose=2)
print(f"Validation {model.metrics_names}: {scores}")

2000/2000 - 30s - loss: 0.1363 - accuracy: 0.9973 - binary_accuracy: 0.9621 - auc_39: 0.6779 - precision_39: 0.3964 - recall_39: 0.0046
Test ['loss', 'accuracy', 'binary_accuracy', 'auc_39', 'precision_39', 'recall_39']: [0.13627831637859344, 0.9973115921020508, 0.9621403813362122, 0.6779302954673767, 0.3964497148990631, 0.004621326923370361]
1646/1646 - 25s - loss: 0.1308 - accuracy: 0.9937 - binary_accuracy: 0.9629 - auc_39: 0.7145 - precision_39: 0.4759 - recall_39: 0.0076
Validation ['loss', 'accuracy', 'binary_accuracy', 'auc_39', 'precision_39', 'recall_39']: [0.13083255290985107, 0.993733286857605, 0.9629223942756653, 0.7145265936851501, 0.47593581676483154, 0.007602938450872898]


In [98]:
y_valid_pred = model.predict(X_valid)
y_test_pred = model.predict(X_test)
y_valid_pred = tf.where(y_valid_pred >= 0.5, 1, 0)
y_test_pred = tf.where(y_test_pred >= 0.5, 1, 0)

print(f"Validation scores:\n{classification_report(y_valid, y_valid_pred, target_names=categories)}")
print(f"Test scores:\n{classification_report(y_test, y_test_pred, target_names=categories)}")

Validation scores:
               precision    recall  f1-score   support

        toxic       0.48      0.02      0.03      5083
 severe_toxic       0.00      0.00      0.00       526
      obscene       0.00      0.00      0.00      2831
       threat       0.00      0.00      0.00       152
       insult       0.00      0.00      0.00      2643
identity_hate       0.00      0.00      0.00       471

    micro avg       0.48      0.01      0.01     11706
    macro avg       0.08      0.00      0.01     11706
 weighted avg       0.21      0.01      0.01     11706
  samples avg       0.00      0.00      0.00     11706

Test scores:
               precision    recall  f1-score   support

        toxic       0.40      0.01      0.02      6090
 severe_toxic       0.00      0.00      0.00       367
      obscene       0.00      0.00      0.00      3691
       threat       0.00      0.00      0.00       211
       insult       0.00      0.00      0.00      3427
identity_hate       0.00     

/home/anto/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anto/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anto/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anto/.local/lib/python3.8/site-packages/sklearn/

In [ ]:
for y, y_hat in zip(y_valid, y_valid_pred):
    print(y, y_hat.numpy())